In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**This project serves as my assignment for the courses below**
<br></br>
[Google Advanced Data Analytics Professional Certificate](https://www.coursera.org/professional-certificates/google-advanced-data-analytics)

In [ ]:
#data manipulation
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns' ,200)

#data modelleing
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Metrics 
from sklearn.model_selection import GridSearchCV ,train_test_split
from sklearn.metrics import (accuracy_score , precision_score , f1_score ,recall_score, \
                            classification_report , confusion_matrix , ConfusionMatrixDisplay, \
                            roc_auc_score , roc_curve)
from sklearn.tree import plot_tree

import pickle


In [ ]:
data=pd.read_csv('/kaggle/input/hr-analytics-and-job-prediction/HR_comma_sep.csv')
data.head()

<a id='data_cleaning'> </a>
# Data Cleaning

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe().T

*Tenure seems to have outliers at high end of Distribution , Maximum value is much higher as compared to other Quartile values*

In [ ]:
data.columns

In [ ]:
data= data.rename(columns = { 'average_montly_hours' : 'average_monthly_hours',\
             'time_spend_company' : 'tenure'})

In [ ]:
print("Unique Values By Features")
for feature in data.columns:
    uniq = np.unique(data[feature])
    print(feature,len(uniq))

In [ ]:
data.isna().sum()

In [ ]:
categorical_columns = set(data.columns) - set(data._get_numeric_data().columns)

for cat in categorical_columns:
    print(cat , data[cat].apply(lambda x:len(x.strip()) == 0 or x.strip().lower() == 'nan').sum())

In [ ]:
data.duplicated().sum()

In [ ]:
data[data.duplicated].head()

In [ ]:
data_1 = data.drop_duplicates(keep = 'first')

In [ ]:
# Eamining tenure in depth 
def boxplott (colum):
    plt.figure(figsize = (6,6))
#     plt.title()
    sns.boxplot(x=data_1[colum])
    plt.show()

In [ ]:
boxplott('tenure')

In [ ]:
boxplott('average_monthly_hours')

In [ ]:
boxplott('number_project')

In [ ]:
def outliers_check (colum):
    Q1 = data_1[colum].quantile(0.25)
    Q3 = data_1[colum].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 -1.5 * IQR
    upper_limit= Q3 + 1.5 * IQR
    print('Lower Limit :' , lower_limit)
    print('upper Limit :' , upper_limit)
    
    outliers = data_1[(data_1['tenure'] > upper_limit) | (data_1['tenure'] < lower_limit) ]
    print('Number of outliers : ' , len (outliers))
    
    

In [ ]:
Q1 = data_1['tenure'].quantile(0.25)    
Q3 = data_1['tenure'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 -1.5 * IQR
upper_limit= Q3 + 1.5 * IQR
# print('Lower Limit :' , lower_limit)
# print('upper Limit :' , upper_limit)
    
outliers = data_1[(data_1['tenure'] >= upper_limit) | (data_1['tenure'] <= lower_limit) ]
print('Number of outliers : ' , len (outliers))

In [ ]:
outliers_check ('tenure')

It is to be noted that yiu consider the outliers depending upon which ML model is going to be used eg Losgic Regrestion is sensetive towards outliers while XGBoost is robust towards outliers.

# Visualization

In [ ]:

fig , ax = plt.subplots(1,2 , figsize = (22,8))

#ax[0] boxplot showing 'average _monthly_hours' distribustions for "number of projects"
sns.boxplot(data = data_1 , x=('average_monthly_hours'), y = ('number_project'),hue = 'left',\
           orient = 'h' , ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_title('Monthly hours by number of projects' , fontsize ='14')


#histogram
# tenure_stay =data_1[data_1['left']==0]['number_project']
# tenure_left =data_1[data_1['left']==1]['number_project']
sns.histplot(data=data_1 , x='number_project', hue= 'left' , multiple='dodge' , shrink=2 ,ax=ax[1])
ax[1].set_title('Number of Projects' , fontsize = '14')

plt.show()

So more projects ultimately means longer work hours .
The ppoint of concern is why people with 7 projects left 

##### 

In [ ]:
data_1.loc[data_1['number_project']==7]["left"].sum()

All the employees with 7 projects indeed left the company

In [ ]:
data_1["salary"].info()

# Logistic REgression

In [ ]:
data_encode = data_1.copy()
data_encode['salary'] = data_encode['salary'].astype('category').cat.set_categories(['low','medium','high']).cat.codes


data_encode = pd.get_dummies(data_encode , drop_first = False).astype(int)
data_encode.head()

In [ ]:
data_encode.columns

In [ ]:
sns.heatmap(data_encode[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_monthly_hours', 'tenure']].corr() , annot=True , cmap='crest'  )

In [ ]:
data_logreg = data_encode[(data_encode['tenure'] <= upper_limit) & (data_encode['tenure'] >=lower_limit)]

In [ ]:
data_logreg.head()

In [ ]:
# Separtin X and y variables

y=data_logreg['left']
X =data_logreg.drop('left' , axis=1)

In [ ]:
y.value_counts(normalize = True)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y ,test_size=0.25 , stratify =y , random_state=42)

In [ ]:
log_reg = LogisticRegression(max_iter=500).fit(X_train,y_train)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test ,y_pred , labels = log_reg.classes_)
cm_display = ConfusionMatrixDisplay(confusion_matrix= cm ,  display_labels=log_reg.classes_)
cm_display.plot()
plt.show()

# Tree Basaed Model

In [ ]:
y=data_encode['left']
X=data_encode.drop('left' , axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.25 , stratify=y , random_state =42)

In [ ]:
dt_clf1 = DecisionTreeClassifier()

In [ ]:
# export_train_test = pd.concat([X_train , y_train] , axis=1 )
# export_train_test.to_csv('train_df' , index = False)

In [ ]:
# While choosing max_depth you have to careful as large value will result in overfitting and small value underfitting\
# so choose a middle value

#For min_sample_split the greater value it's more prone to underfitting the lower value will result in overfittiing


cv_params = {
    'max_depth': [4, 6, 8, None],  # Corrected parameter name
    'min_samples_split': [2, 4, 6],  # Corrected parameter name
    'min_samples_leaf': [2, 5, 1]
}

scoring={ 'accuracy','precision' ,'recall','f1','roc_auc'}

dt_clf1=GridSearchCV (dt_clf1 , param_grid = cv_params, scoring=scoring , cv=4 ,refit='roc_auc')

In [ ]:
%%time
dt_clf1.fit(X_train,y_train)

In [ ]:
dt_clf1.best_params_

In [ ]:
#AUC score
dt_clf1.best_score_

In [ ]:
# !pip install autopip8

In [ ]:
def make_result (model_name:str ,gscv_object , metric:str):
    metric_dict = {"precision" : "mean_test_precision", \
              "recall" :"mean_test_recall",
              "f1" : "mean_test_f1",
              "accuracy" : "mean_test_accuracy",
              "auc" :"mean_test_roc_auc"}
    cv_results =pd.DataFrame(gscv_object.cv_results_)  
    best_estimator_results =cv_results.iloc[cv_results[metric_dict[metric]].idxmax(),:]


    auc=best_estimator_results.mean_test_roc_auc
    f1=best_estimator_results.mean_test_f1
    recall=best_estimator_results.mean_test_recall
    precision=best_estimator_results.mean_test_precision
    accuracy=best_estimator_results.mean_test_accuracy

    table=pd.DataFrame()
    table=pd.DataFrame({'model' : [model_name],
                      'precision' : [precision],
                     'recall':[recall],
                      'accuracy': [accuracy],
                      'auc': [auc]})

    return table


In [ ]:
dt_clf1_results = make_result('decision tree cv' , dt_clf1,'auc')
dt_clf1_results

In [ ]:

rf = RandomForestClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth': [3,5, None], 
             'max_features': [1.0],
             'max_samples': [0.7, 1.0],
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'n_estimators': [300, 500],
             }  

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
rf1 = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='roc_auc')

In [ ]:
%%time
rf1.fit(X_train, y_train)

In [ ]:
path = '/kaggle/working/'

In [ ]:
def write_pickle(path, model_object, save_as:str):


    with open(path + save_as + '.pickle', 'wb') as to_write:
        pickle.dump(model_object, to_write)

In [ ]:
def read_pickle(path, saved_model_name:str):
    '''
    In: 
        path:             path to folder where you want to read from
        saved_model_name: filename of pickled model you want to read in

    Out: 
        model: the pickled model 
    '''
    with open(path + saved_model_name + '.pickle', 'rb') as to_read:
        model = pickle.load(to_read)

    return model

In [ ]:
write_pickle(path, rf1, 'hr_rf1')

In [ ]:
rf1 = read_pickle(path, 'hr_rf1')

In [ ]:
rf1.best_score_

In [ ]:
rf1.best_params_

In [ ]:
rf1_cv_results = make_result('random forest cv', rf1, 'auc')
print(dt_clf1_results)
print(rf1_cv_results)

In [ ]:
def get_scores(model_name:str, model, X_test_data, y_test_data):
    '''
    Generate a table of test scores.

    In: 
        model_name (string):  How you want your model to be named in the output table
        model:                A fit GridSearchCV object
        X_test_data:          numpy array of X_test data
        y_test_data:          numpy array of y_test data

    Out: pandas df of precision, recall, f1, accuracy, and AUC scores for your model
    '''

    preds = model.best_estimator_.predict(X_test_data)

    auc = roc_auc_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision], 
                          'recall': [recall],
                          'f1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                         })
  
    return table

In [ ]:
rf1_test_scores = get_scores('random forest1 test', rf1, X_test, y_test)
rf1_test_scores

In [ ]:
# Drop `satisfaction_level` and save resulting dataframe in new variable
df2 = data_encode.drop('satisfaction_level', axis=1)

# Display first few rows of new dataframe
df2.head()

In [ ]:
# Create `overworked` column. For now, it's identical to average monthly hours.
df2['overworked'] = df2['average_monthly_hours']

# Inspect max and min average monthly hours values
print('Max hours:', df2['overworked'].max())
print('Min hours:', df2['overworked'].min())

In [ ]:
# Define `overworked` as working > 175 hrs/week
df2['overworked'] = (df2['overworked'] > 175).astype(int)

# Display first few rows of new column
df2['overworked'].head()

In [ ]:
# Drop the `average_monthly_hours` column
df2 = df2.drop('average_monthly_hours', axis=1)

# Display first few rows of resulting dataframe
df2.head()

In [ ]:
# Isolate the outcome variable
y = df2['left']

# Select the features
X = df2.drop('left', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

In [ ]:
# Instantiate model
tree = DecisionTreeClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth':[4, 6, 8, None],
             'min_samples_leaf': [2, 5, 1],
             'min_samples_split': [2, 4, 6]
             }

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
tree2 = GridSearchCV(tree, cv_params, scoring=scoring, cv=4, refit='roc_auc')

In [ ]:
%%time
tree2.fit(X_train, y_train)

In [ ]:
tree2.best_params_

In [ ]:
tree2.best_score_

In [ ]:
tree2_cv_results = make_result('decision tree2 cv', tree2, 'auc')
print(dt_clf1_results)
print(tree2_cv_results)

In [ ]:
rf = RandomForestClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth': [3,5, None], 
             'max_features': [1.0],
             'max_samples': [0.7, 1.0],
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'n_estimators': [300, 500],
             }  

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
rf2 = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='roc_auc')

In [ ]:
%%time
rf2.fit(X_train, y_train)

In [ ]:
write_pickle(path, rf2, 'hr_rf2')

In [ ]:
rf2 = read_pickle(path, 'hr_rf2')

In [ ]:
rf2.best_params_

In [ ]:
rf2.best_score_

In [ ]:
# Get all CV scores
rf2_cv_results = make_result('random forest2 cv', rf2, 'auc')
print(tree2_cv_results)
print(rf2_cv_results)

In [ ]:
# Get predictions on test data
rf2_test_scores = get_scores('random forest2 test', rf2, X_test, y_test)
rf2_test_scores

In [ ]:
# Generate array of values for confusion matrix
preds = rf2.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, preds, labels=rf2.classes_)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=rf2.classes_)
disp.plot(values_format='');

In [ ]:
# Plot the tree
plt.figure(figsize=(85,20))
plot_tree(tree2.best_estimator_, max_depth=6, fontsize=14, feature_names=X.columns, 
          class_names={0:'stayed', 1:'left'}, filled=True);
plt.show()

In [ ]:
#tree2_importances = pd.DataFrame(tree2.best_estimator_.feature_importances_, columns=X.columns)
tree2_importances = pd.DataFrame(tree2.best_estimator_.feature_importances_, 
                                 columns=['gini_importance'], 
                                 index=X.columns
                                )
tree2_importances = tree2_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
tree2_importances = tree2_importances[tree2_importances['gini_importance'] != 0]
tree2_importances

In [ ]:
sns.barplot(data=tree2_importances, x="gini_importance", y=tree2_importances.index, orient='h')
plt.title("Decision Tree: Feature Importances for Employee Leaving", fontsize=12)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

In [ ]:
# Get feature importances
feat_impt = rf2.best_estimator_.feature_importances_

# Get indices of top 10 features
ind = np.argpartition(rf2.best_estimator_.feature_importances_, -10)[-10:]

# Get column labels of top 10 features 
feat = X.columns[ind]

# Filter `feat_impt` to consist of top 10 feature importances
feat_impt = feat_impt[ind]

y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
y_sort_df = y_df.sort_values("Importance")
fig = plt.figure()
ax1 = fig.add_subplot(111)

y_sort_df.plot(kind='barh',ax=ax1,x="Feature",y="Importance")

ax1.set_title("Random Forest: Feature Importances for Employee Leaving", fontsize=12)
ax1.set_ylabel("Feature")
ax1.set_xlabel("Importance")

plt.show()

*Logistic Regression

Logistic Regression model yielded impressive results: precision 80%, recall 83%, and f1-score 80%, with an accuracy of 83% on the test set.
Tree-based Machine Learning

Enhanced decision tree model achieved AUC 93.8%, precision 87.0%, recall 90.4%, and f1-score 88.7%, with an outstanding accuracy of 96.2% on the test set. Random Forest slightly outperformed.**